In [1]:
from rdkit import Chem
from mordred import Calculator, descriptors
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import readline
%matplotlib inline

In [2]:
calc = Calculator(descriptors,ignore_3D=True)

In [3]:
print "Total number of 2D and 3D descriptors:",len(calc.descriptors)

Total number of 2D and 3D descriptors: 1612


In [4]:
print "Total number of 2D descriptors",len(calc.descriptors)

Total number of 2D descriptors 1612


# Load SMILES strings

In [95]:
#kegg_mets = pd.read_csv('All_ChemDes_KEGG_inputs.tab',sep='\t')
#kegg_mets
cyp_drugs = pd.read_excel('3_CYP_to_SMILES_curated.xlsx')
# For some reason, imatinib (drug in row 478) doesn't work with RDkit so, we will remove it
cyp_drugs = cyp_drugs[cyp_drugs['Drug Identifier']!='imatinib']
cyp_drugs.index = range(len(cyp_drugs))
cyp_drugs.head()

,Drug Identifier,PUBCHEM,SMILES
0,aminobutyric acid,119.0,C(CC(=O)O)CN
1,acetic acid,176.0,CC(=O)O
2,acetylcholin,187.0,CC(=O)OCC[N+](C)(C)C
3,acetylcholine,187.0,CC(=O)OCC[N+](C)(C)C
4,bupropion,444.0,CC(C(=O)C1=CC(=CC=C1)Cl)NC(C)(C)C


In [8]:
kegg_drugs = pd.read_csv('All_ChemDes_KEGG_DRUG_inputs.tab',sep='\t')
kegg_drugs

,KEGG,CAS,PUBCHEM,SMILES
0,dr:D06978,NaN,51091320.0,C#CCCCOC(=O)C1=CC(=C(C=C1)N2C=NC=N2)[N+](=O)[O-]
1,dr:D06980,NaN,51091322.0,C#CCCOC(=O)C1=CC(=C(C=C1)N2C=NC=N2)[N+](=O)[O-]
2,dr:D03934,80263-73-6,47205922.0,C#CCN1CCN(CC1)C(=O)C2=CC3=CC=CC=C3O2
3,dr:D03939,292819-64-8,47205927.0,C#CCN1CCN(CC1)C2=NC=C(C=N2)Br
4,dr:D04025,366017-09-6,17398009.0,C#CCNC(=O)N1CCN(CC1)C2=CC=C(C=C2)F
5,dr:D03463,77146-42-0,17397602.0,C/C=C/C1=C(C(=CC(=C1)C(=O)NC2=CC=CC(=C2)C(=O)C...
6,dr:D03203,332348-12-6,17397357.0,C/C=C/C1=C(C(=CC(=C1)C(=O)OCC(=O)C2=C(C(=C(N2)...
7,dr:D03877,444190-52-7,17397961.0,C/C=C/C=C/C(=O)NC1=CC(=C(C=C1)OC)OC
8,dr:D03876,NaN,17397960.0,C/C=C/C=C/C(=O)NC1=CC=C(C=C1)N2CCOCC2
9,dr:D07162,6903-79-3,51091501.0,C/C=C/C=C/C(=O)NCCCN1C(=O)N2C=CC=CC2=N1


In [9]:
smiles = list(kegg_drugs['SMILES'])
#smiles = list(cyp_drugs['SMILES'])

In [10]:
molecules = [Chem.MolFromSmiles(smi) for smi in smiles]

In [11]:
# Get column names for a few molecules
cols = list(calc.pandas(molecules[475:480]).columns)

100%|██████████| 5/5 [00:00<00:00,  7.16it/s]


In [ ]:
# Calculate Molecular descriptors in numeric form
dat = np.zeros([len(molecules),len(cols)])
for mol in range(len(molecules)):
    tmp_mds = calc(molecules[mol])
    mds = np.zeros([1,len(tmp_mds)])
    for i in range(len(tmp_mds)):
        mds[0,i] = float(tmp_mds[i])
    dat[mol,:] = mds

In [102]:
MDs = pd.DataFrame(columns=cols,data=dat)

# Calculate Molecular descriptors from SMILES

In [17]:
molecule = Chem.MolFromSmiles(sn38_smiles)
tmp_mds = calc(molecule)
mds = np.zeros([1,len(tmp_mds)])
dat = np.zeros([1,len(cols)])
for i in range(len(tmp_mds)):
    mds[0,i] = float(tmp_mds[i])
    dat[0,:] = mds

In [18]:
MDs = pd.DataFrame(columns=cols,data=dat)

In [19]:
MDs

,SpAbs_D,SpMax_D,SpDiam_D,SpAD_D,SpMAD_D,LogEE_D,SM1_D,VE1_D,VE2_D,VE3_D,...,EState_VSA10,VSA_EState1,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,VSA_EState9
0,280.860719,140.430359,204.988038,280.860719,9.684852,140.430359,1.402605e-13,5.306952,0.182998,2.733728,...,19.802129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.628604,70.454729


In [103]:
test.head()

,Drug Identifier,PUBCHEM,SMILES,SpAbs_D,SpMax_D,SpDiam_D,SpAD_D,SpMAD_D,LogEE_D,SM1_D,...,EState_VSA10,VSA_EState1,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,VSA_EState9
0,aminobutyric acid,119.0,C(CC(=O)O)CN,30.809534,15.404767,24.297933,30.809534,4.401362,15.404767,1.998401e-14,...,4.794537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,23.166667
1,acetic acid,176.0,CC(=O)O,9.291503,4.645751,6.645751,9.291503,2.322876,4.662840,1.110223e-15,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,16.666667
2,acetylcholin,187.0,CC(=O)OCC[N+](C)(C)C,57.334033,28.667017,44.938889,57.334033,5.733403,28.667017,-2.176037e-14,...,4.794537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.826667,23.840000
3,acetylcholine,187.0,CC(=O)OCC[N+](C)(C)C,57.334033,28.667017,44.938889,57.334033,5.733403,28.667017,-2.176037e-14,...,4.794537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.826667,23.840000
4,bupropion,444.0,CC(C(=O)C1=CC(=CC=C1)Cl)NC(C)(C)C,116.958704,58.479352,87.459206,116.958704,7.309919,58.479352,1.136868e-13,...,4.794537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,31.340371
